In [2]:
# Loading libraries for the course
import random
import matplotlib.pyplot as plt             # Library used for plotting
from matplotlib.patches import Rectangle    # Module to plot a rectangle in the image
import urllib.request                       # Library to download data
import numpy as np                          # Library for array manipulation
import seaborn as sn                        # Library for advanced plotting
import pandas as pd                         # Library to manipulate data frames
import tifffile                             # Library to store numpy arrays as TIFF
import pathlib                              # Library to work with file paths
from pathlib import Path                    # Library to work with file paths
import skimage                              # Library for image manipulation. scikit-image
from skimage.io import imread               # Module from skimage
from matplotlib import animation            # Module to plot animations
from skimage import filters, measure, segmentation, color
from scipy.ndimage import gaussian_filter
from skimage.filters import difference_of_gaussians
import pandas as pd
from cellpose import models
from scipy import ndimage as ndi

In [3]:
import tifffile as tiff
import numpy as np
import pandas as pd
import cv2

# Read the multi-frame TIF file
figName = '../Drug7_1000nM/rep1_FOV1.tif'
img = tiff.imread(figName)

use_GPU = True

# Initialize the Cellpose model
model = models.Cellpose(gpu=use_GPU, model_type='cyto')

#createDataFrame
columns = ['frame', 'cell_id', 'mean_intensity', 'area']
df = pd.DataFrame(columns=columns)

first_frame = img[0]
if first_frame.ndim == 3:
    first_frame = first_frame[1]

# Segment cells using Cellpose on the first frame
masks, flows, styles, diams = model.eval(first_frame, diameter=150, channels=[0, 0])

# Verify the shape of the image stack
print(img.shape)  # (num_frames, height, width)

results = []

for frame_idx, frame in enumerate(img):
    print(f"Processing frame {frame_idx}...")

     # If the frame has multiple channels, use only the first channel
    if frame.ndim == 3:
        frame = frame[1]
    
    # Get unique cell masks
    unique_masks = np.unique(masks)
    
    # Loop through each unique mask (cell)
    for cell_id in unique_masks:
        if cell_id == 0:
            continue  # Skip background
        
        # Create a mask for the current cell
        cell_mask = (masks == cell_id).astype(np.uint8)
        
        # Check the sizes of the frame and cell_mask
        if frame.shape != cell_mask.shape:
            print(f"Size mismatch: frame shape {frame.shape}, cell_mask shape {cell_mask.shape}")
            continue
        
        # Calculate the mean intensity of the current cell
        mean_intensity = cv2.mean(frame, mask=cell_mask)[0]
        
        # Calculate the area of the current cell
        area = np.sum(cell_mask)
        
        # Append the results to the list
        results.append({
            'frame': frame_idx,
            'cell_id': cell_id,
            'mean_intensity': mean_intensity,
            'area': area
        })

# Convert the results list to a DataFrame
df = pd.DataFrame(results)

# Print the DataFrame
print(df)

(21, 3, 650, 650)
Processing frame 0...
Processing frame 1...
Processing frame 2...
Processing frame 3...
Processing frame 4...
Processing frame 5...
Processing frame 6...
Processing frame 7...
Processing frame 8...
Processing frame 9...
Processing frame 10...
Processing frame 11...
Processing frame 12...
Processing frame 13...
Processing frame 14...
Processing frame 15...
Processing frame 16...
Processing frame 17...
Processing frame 18...
Processing frame 19...
Processing frame 20...
     frame  cell_id  mean_intensity  area
0        0        1       89.183630  7575
1        0        2       72.710237  8010
2        0        3       64.478067  7386
3        0        4       97.236349  7252
4        0        5       83.329544  8005
..     ...      ...             ...   ...
289     20       10       39.874803  8251
290     20       11       64.443256  7666
291     20       12       64.136892  8262
292     20       13       65.581799  8362
293     20       14       64.551637  7543

[294